# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df= pd.read_csv('../output_data/weather_df.csv')
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,50.09,81,80,4.07,AU,1636054455
1,Gibara,21.1097,-76.1317,90.72,70,42,11.79,CU,1636054455
2,Mataura,-46.1927,168.8643,52.48,65,100,0.45,NZ,1636054455
3,Ribeira Grande,38.5167,-28.7000,70.20,88,75,24.16,PT,1636054456
4,Morón,-34.6534,-58.6198,85.01,61,51,1.99,AR,1636054259
...,...,...,...,...,...,...,...,...,...
531,Samfya,-11.3649,29.5565,75.92,51,4,6.26,ZM,1636054614
532,Stornoway,58.2093,-6.3865,49.96,93,75,11.50,GB,1636054614
533,Aktash,50.3000,87.7333,-16.94,84,15,2.59,RU,1636054615
534,Berlin,52.5244,13.4105,47.14,93,90,31.00,DE,1636054382


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
locations= weather_df[['Lat','Lng']]#.astype(float)

# Get the humidity rating
humidity= weather_df['Humidity']#.astype(float)

# Plot Heatmap, create a gmaps figure 
fig= gmaps.figure(center=(17.0, 4.0), zoom_level=2)

#create a heat layer (search the gmaps documentation - hint there's a heatmap_layer method)
heat_layer= gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=4)

#add the layer to your figure
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
nice_weather_df = weather_df.loc[(weather_df['Max Temp'] >70) &
                          (weather_df['Max Temp'] <80) &
                          (weather_df['Wind Speed'] <10) &
                          (weather_df['Cloudiness'] ==0)].dropna()
                     

nice_weather_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
150,Grand Gaube,-20.0064,57.6608,72.00,76,0,4.00,MU,1636054496
199,Makkah al Mukarramah,21.4267,39.8261,79.92,61,0,3.02,SA,1636054485
232,Riyadh,24.6877,46.7219,73.54,21,0,4.50,SA,1636054525
262,Cap Malheureux,-19.9842,57.6142,72.09,76,0,4.00,MU,1636054533
308,Sabha,27.0377,14.4283,79.05,15,0,7.78,LY,1636054323
317,Waddān,29.1614,16.1390,79.27,20,0,5.26,LY,1636054548
325,Nizwá,22.9333,57.5333,71.74,47,0,2.44,OM,1636054551
341,Awjilah,29.1081,21.2869,79.11,25,0,8.66,LY,1636054556
463,Ibrā’,22.6906,58.5334,70.59,55,0,3.29,OM,1636054594
495,Kiryat Gat,31.6100,34.7642,71.40,73,0,6.60,IL,1636054382


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df[["City","Country","Lat","Lng"]]
hotel_df['Hotel Name'] = ""
hotel_df.head(10)

<ipython-input-8-6a610bfcd4e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Hotel Name
150,Grand Gaube,MU,-20.0064,57.6608,
199,Makkah al Mukarramah,SA,21.4267,39.8261,
232,Riyadh,SA,24.6877,46.7219,
262,Cap Malheureux,MU,-19.9842,57.6142,
308,Sabha,LY,27.0377,14.4283,
317,Waddān,LY,29.1614,16.1390,
325,Nizwá,OM,22.9333,57.5333,
341,Awjilah,LY,29.1081,21.2869,
463,Ibrā’,OM,22.6906,58.5334,
495,Kiryat Gat,IL,31.6100,34.7642,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()

    results= response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except:
        hotel_df.loc[index, "Hotel Name"] = "Hotel Not Found"
        
hotel_df

C:\Users\KatharineHollars\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
150,Grand Gaube,MU,-20.0064,57.6608,LUX Grand Gaube Resort & Villas
199,Makkah al Mukarramah,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
232,Riyadh,SA,24.6877,46.7219,"Radisson Blu Hotel, Riyadh"
262,Cap Malheureux,MU,-19.9842,57.6142,LUX Grand Gaube Resort & Villas
308,Sabha,LY,27.0377,14.4283,فندق المواجيد
317,Waddān,LY,29.1614,16.1390,Hotel Not Found
325,Nizwá,OM,22.9333,57.5333,Antique Inn - Nizwa
341,Awjilah,LY,29.1081,21.2869,Hotel Not Found
463,Ibrā’,OM,22.6906,58.5334,Ibra Hotel
495,Kiryat Gat,IL,31.6100,34.7642,Desert Gat


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))